จาก ep ก่อน ที่เราสอนเรื่อง [ใช้ Deep Neural Network วิเคราะห์ข้อมูลแบบ Structure หรือข้อมูลแบบตาราง Tabular Data ](https://www.bualabs.com/archives/958/machine-learning-deep-learning-structure-data-tabular-data-ep-1/) จะมีงานสำคัญที่ต้องทำก่อนเทรน คือการ Preprocessing จัดเตรียมข้อมูล ซึ่งมีหลายอย่าง เช่น Fillna, Normalize, Categorize, Clipping, Binning, Feature Engineering ต่าง ๆ , etc.  ใน ep นี้เราจะมาเรียนรู้กันเรื่อง การจัดการกับข้อมูลไม่ครบ วิธีเติมค่าที่ขาดหายไป หรือ Null, สอน FillNA - Preprocessing ep.1 

(NA = Not Available, Null = ไม่มีข้อมูล, NaN = Not a Number, Null = ไม่มีค่า)

# 0. Magic Commands

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# 1. Import Library

In [0]:
from fastai import *
from fastai.tabular import *

# 2. Data

In [0]:
path = untar_data(URLs.ADULT_SAMPLE)
# df = DataFrame, pd = Pandas
df = pd.read_csv(path/'adult.csv')

## 2.1 สำรวจข้อมูล

เปิดดู 10 Row แรก สังเกตว่า ใน Column occupation และ education-num บาง Row จะมีค่าเป็น NaN

In [22]:
df.head(10)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k
5,20,Private,63210,HS-grad,9.0,Never-married,Handlers-cleaners,Own-child,White,Male,0,0,15,United-States,<50k
6,49,Private,44434,Some-college,10.0,Divorced,NaN,Other-relative,White,Male,0,0,35,United-States,<50k
7,37,Private,138940,11th,7.0,Married-civ-spouse,NaN,Husband,White,Male,0,0,40,United-States,<50k
8,46,Private,328216,HS-grad,9.0,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,>=50k
9,36,Self-emp-inc,216711,HS-grad,NaN,Married-civ-spouse,NaN,Husband,White,Male,99999,0,50,?,>=50k


## 2.2 ดูสถิติข้อมูล

เราลองมาดูแบบสรุป ว่ามี Column ไหนบ้างที่ข้อมูลขาด ถ้า Column ไหน Count ไม่เท่ากับ 32561 แสดงว่ามีข้อมูลขาดหายไป

In [24]:
df.describe(include='all')

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
count,32561.000000,32561,3.256100e+04,32561,32074.000000,32561,32049,32561,32561,32561,32561.000000,32561.000000,32561.000000,32561,32561
unique,NaN,9,NaN,16,NaN,7,15,6,5,2,NaN,NaN,NaN,42,2
top,NaN,Private,NaN,HS-grad,NaN,Married-civ-spouse,Prof-specialty,Husband,White,Male,NaN,NaN,NaN,United-States,<50k
freq,NaN,22696,NaN,10501,NaN,14976,4073,13193,27816,21790,NaN,NaN,NaN,29170,24720
mean,38.581647,NaN,1.897784e+05,NaN,10.079815,NaN,NaN,NaN,NaN,NaN,1077.648844,87.303830,40.437456,NaN,NaN
std,13.640433,NaN,1.055500e+05,NaN,2.572999,NaN,NaN,NaN,NaN,NaN,7385.292085,402.960219,12.347429,NaN,NaN
min,17.000000,NaN,1.228500e+04,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,1.000000,NaN,NaN
25%,28.000000,NaN,1.178270e+05,NaN,9.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,40.000000,NaN,NaN
50%,37.000000,NaN,1.783560e+05,NaN,10.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,40.000000,NaN,NaN
75%,48.000000,NaN,2.370510e+05,NaN,12.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,45.000000,NaN,NaN


ในที่นี้คือ occupation (count) = 32049, education-num (count) = 32074

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
age               32561 non-null int64
workclass         32561 non-null object
fnlwgt            32561 non-null int64
education         32561 non-null object
education-num     32074 non-null float64
marital-status    32561 non-null object
occupation        32049 non-null object
relationship      32561 non-null object
race              32561 non-null object
sex               32561 non-null object
capital-gain      32561 non-null int64
capital-loss      32561 non-null int64
hours-per-week    32561 non-null int64
native-country    32561 non-null object
salary            32561 non-null object
dtypes: float64(1), int64(5), object(9)
memory usage: 3.7+ MB


df DataFrame เรามีทั้งหมด 32561 Row x 15 Column

# 3. Prepare Data

ในโลกแห่งความเป็นจริง เราจะพบปัญหาข้อมูลขาดหายไปอยู่เสมอ เช่น User กรอกแบบฟอร์มไม่ครบ หรือ ใส่ - ใส่ space มาแทน, ระบบมีหลายเวอร์ชัน, ข้อมูลมาจากหลายที่ ที่ Format แตกต่างกันนิดหน่อย, etc.

เมื่อมีข้อมูลขาดหายไป เราจะเป็นต้องจัดการข้อมูลเหล่านั้น ก่อนป้อนเข้าสู่ระบบ Neural Network ก่อนจะใช้เทรน Machine Learning เนื่องจากโมเดลจะรับข้อมูลเป็นตัวเลขเท่านั้น การรับข้อมูล Null, NaN ทำให้มีผลต่อการคำนวน สูตรคณิตศาสร์ภายในโมเดล เช่น NaN * 5 = ? หรือ หาค่าเฉลี่ย แล้วมีค่า NaN ปนมาด้วย ทำให้รวนกันไปหมด


วิธีจัดการค่า Null / NaN

1. Drop Row นั้นทิ้งไปเลย
1. Drop Column นั้นทิ้งไปเลย
1. Imputation เติมด้วยค่าพิเศษ ที่ไม่มีอยู่ในข้อมูล เช่นตัวเลข -1, -Infinity หรือข้อความว่า "Unknown", "N/A"
1. Imputation เติมด้วยค่า Mean / Median / Mode จากข้อมูลทั้งหมด เช่น เติมอายุที่ขาดไป จากอายุเฉลี่ยของข้อมูลทั้ง Dataset
1. Imputation เติมด้วยค่า Mean / Median / Mode จากข้อมูลบางส่วน เช่น เติมอายุที่ขาดไป จากอายุเฉลี่ยของนักเรียนห้องเดียวกัน
1. Imputation เติมด้วยค่าจากอัลกอริทึม เช่น K-Nearest Neighbour, Bi-Linear Interpolation, etc.
1. Imputation เติมด้วยค่าด้วยมือของเราเอง ทีละค่า ๆ ด้วยการแก้ไขข้อมูลโดยตรง จากวิจารณญาณ
1. เติมด้วยค่าที่ต่อเนื่องก่อนหน้า หรือค่าถัดไป สำหรับข้อมูลแบบ Time Series หรือข้อมูลมีลำดับ
1. Linear Interpolation + Seasonal Adjustment สำหรับข้อมูลแบบ Time Series หรือข้อมูลมีลำดับ
1. สร้าง Category ใหม่ ถือว่าค่า Null, NaN ก็เป็นข้อมูล 1 Category
1. ใช้หลาย ๆ วิธีผสมกัน แล้วหาค่าเฉลี่ย, Mode
1. Preprocessing ไปแล้ว เทรนโมเดลเรียบร้อย แล้วนำผลลัพธ์กลับมาปรับจูน Preprocessing รอบใหม่

Mean = ค่าเฉลี่ย, Median = ค่ามัธยฐาน, Mode = ค่าฐานนิยม (ค่าที่พบมากที่สุด) สำหรับข้อมูลตัวเลข เราสามาถเลือกใช้ได้ทั้ง 3 อย่าง แต่ถ้าเป็นข้อมูลข้อความ หรือ Category เราต้องใช้ Mode แทน




ทุกวิธีล้วนมีข้อดีข้อเสียแตกต่างกัน ตั้งแต่ Effort ที่ใช้, เวลาในประมวลผล, คุณภาพของข้อมูล, Reproducibility, ความแม่นยำของโมเดล 

การจะเลือกวิธีใด ควรพิจารณาจาก ชนิดข้อมูล ขนาดของข้อมูล การกระจายของข้อมูล ความเร็วของข้อมูล จำนวนของข้อมูลที่ขาดหายไป บวกกับข้อดีข้อเสียด้านบน

เราจะเลือกวิธีใดถือเป็น Hyperparameter แบบนึง แต่เป็น Indirect Hyperparameter ทางอ้อม เพราะไม่ได้ทำกับโมเดลโดยตรง

หมายเหตุ Imputation = การกำหนดค่าให้บางอย่าง โดยอนุมานเอาจาก ค่าของผลลัพธ์ หรือกระบวนการที่ค่านั้นสนับสนุนอยู่ (the assignment of a value to something by inference from the value of the products or processes to which it contributes.)

## 3.1 Drop Row ที่มี Null ทิ้ง

In [26]:
df2 = df.dropna(axis=0)
df2.describe(include='all')

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
count,31812.000000,31812,3.181200e+04,31812,31812.000000,31812,31812,31812,31812,31812,31812.000000,31812.000000,31812.000000,31812,31812
unique,NaN,9,NaN,16,NaN,7,15,6,5,2,NaN,NaN,NaN,42,2
top,NaN,Private,NaN,HS-grad,NaN,Married-civ-spouse,Prof-specialty,Husband,White,Male,NaN,NaN,NaN,United-States,<50k
freq,NaN,22166,NaN,10264,NaN,14605,4042,12870,27171,21288,NaN,NaN,NaN,28499,24141
mean,38.594367,NaN,1.896966e+05,NaN,10.079341,NaN,NaN,NaN,NaN,NaN,1079.658839,87.268798,40.441311,NaN,NaN
std,13.654887,NaN,1.052524e+05,NaN,2.571659,NaN,NaN,NaN,NaN,NaN,7381.251563,403.054039,12.349341,NaN,NaN
min,17.000000,NaN,1.228500e+04,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,1.000000,NaN,NaN
25%,28.000000,NaN,1.178435e+05,NaN,9.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,40.000000,NaN,NaN
50%,37.000000,NaN,1.783485e+05,NaN,10.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,40.000000,NaN,NaN
75%,48.000000,NaN,2.370545e+05,NaN,12.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,45.000000,NaN,NaN


จะเห็นว่าข้อมูล จาก 32561 Row เหลือแค่ 31812 Row เท่านั้น

## 3.2 Drop Column ที่มี Null ทิ้ง

In [28]:
df3 = df.dropna(axis=1)
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 13 columns):
age               32561 non-null int64
workclass         32561 non-null object
fnlwgt            32561 non-null int64
education         32561 non-null object
marital-status    32561 non-null object
relationship      32561 non-null object
race              32561 non-null object
sex               32561 non-null object
capital-gain      32561 non-null int64
capital-loss      32561 non-null int64
hours-per-week    32561 non-null int64
native-country    32561 non-null object
salary            32561 non-null object
dtypes: int64(5), object(8)
memory usage: 3.2+ MB


จะเห็นว่า จาก 15 Column เหลือแค่ 13 Column เท่านั้น Column occupation และ education-num ถูก Drop หายไปแล้ว

## 3.3 เติมด้วยค่าพิเศษ

In [31]:
values = {'occupation': "UNKNOWN", 'education-num': -1}
df4 = df.fillna(value=values)
df4.head(10)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,UNKNOWN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,-1.0,Divorced,UNKNOWN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,-1.0,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k
5,20,Private,63210,HS-grad,9.0,Never-married,Handlers-cleaners,Own-child,White,Male,0,0,15,United-States,<50k
6,49,Private,44434,Some-college,10.0,Divorced,UNKNOWN,Other-relative,White,Male,0,0,35,United-States,<50k
7,37,Private,138940,11th,7.0,Married-civ-spouse,UNKNOWN,Husband,White,Male,0,0,40,United-States,<50k
8,46,Private,328216,HS-grad,9.0,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,>=50k
9,36,Self-emp-inc,216711,HS-grad,-1.0,Married-civ-spouse,UNKNOWN,Husband,White,Male,99999,0,50,?,>=50k


สังเกตค่า NaN ใน Column occupation กลายเป็น UNKNOWN และ education-num กลายเป็น -1

## 3.4 เติมด้วยค่า Mean, Median, Mode ของ Dataset

หาค่า mean และ mode ที่เราจะ fill จาก Dataset

In [58]:
df5 = df.copy()
val1 = df5["occupation"].mode().max()
val2 = df5["education-num"].median()
val1, val2

(' Prof-specialty', 10.0)

In [59]:
df5["occupation"].fillna(val1, inplace = True)
df5["education-num"].fillna(val2, inplace = True)
df5.head(10)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,Prof-specialty,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,10.0,Divorced,Prof-specialty,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,10.0,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k
5,20,Private,63210,HS-grad,9.0,Never-married,Handlers-cleaners,Own-child,White,Male,0,0,15,United-States,<50k
6,49,Private,44434,Some-college,10.0,Divorced,Prof-specialty,Other-relative,White,Male,0,0,35,United-States,<50k
7,37,Private,138940,11th,7.0,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States,<50k
8,46,Private,328216,HS-grad,9.0,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,>=50k
9,36,Self-emp-inc,216711,HS-grad,10.0,Married-civ-spouse,Prof-specialty,Husband,White,Male,99999,0,50,?,>=50k


สังเกตค่า NaN ใน Column occupation กลายเป็น ' Prof-specialty' และ education-num กลายเป็น 10

## 3.5 เติมด้วยค่า Mean, Median, Mode จากข้อมูลบางส่วน

เติมค่าใน Column education-num ด้วย median ของกลุ่มคนที่ระดับการศึกษาเท่ากัน education เหมือนกัน

In [82]:
df6 = df.copy()
df6["education-num"].fillna(df6.groupby(["education"])["education-num"].transform('median'), inplace = True)
df6.head(10)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,9.0,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,4.0,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k
5,20,Private,63210,HS-grad,9.0,Never-married,Handlers-cleaners,Own-child,White,Male,0,0,15,United-States,<50k
6,49,Private,44434,Some-college,10.0,Divorced,NaN,Other-relative,White,Male,0,0,35,United-States,<50k
7,37,Private,138940,11th,7.0,Married-civ-spouse,NaN,Husband,White,Male,0,0,40,United-States,<50k
8,46,Private,328216,HS-grad,9.0,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,>=50k
9,36,Self-emp-inc,216711,HS-grad,9.0,Married-civ-spouse,NaN,Husband,White,Male,99999,0,50,?,>=50k


สังเกตค่า NaN ใน Column education-num กลายเป็น median ของกลุ่มคนที่ education เหมือนกัน

# 4. สรุป

1. เราสามารถใช้วิธี Drop หรือ Fill NA แบบง่าย ๆ ได้ด้วยโค้ดไม่กี่บรรทัด เนื่องจากเป็นงานที่ต้องทำบ่อย API มีไว้ให้หมดแล้ว แต่ที่สำคัญคือเราจะเลือกใช้วิธีไหน
1. ยิ่งเรารู้จักข้อมูลมากเท่าไร ก็ทำให้เรายิ่งตัดสินใจเลือกวิธีจัดการข้อมูลได้ดีขึ้นเท่านั้น 
1. โมเดลที่ดี ค่อนข้างจะทนทานต่อข้อมูลเสีย Noise เล็ก ๆ น้อย ๆ ฉะนั้นเราไม่ต้องทำข้อมูลให้ Perfect มาก ทำข้อมูลที่ดีพอสมควรให้เร็วที่สุด ป้อนให้โมเดลก่อน แล้วค่อยมาปรับจูนใน Iteration ต่อไป
